# Neuropils and Point Lookups

This tutorial provides instructions for how to download neuropil meshes and quickly calculate in which neuropil a point in space is located.

Note on versions: We updated the neuropil meshes in the optic lobes between versions 630 and 783. We recommend using the later version of the neuropils but both are provided for the scenario that one wants to replicate analyses published with version 630.

## CloudVolume

[CloudVolume](https://github.com/seung-lab/cloud-volume) is a powerful Python library for accessing 3d volumetric data stored in the cloud (Google, AWS). You can install cloudvolume via

```
pip install cloud-volume
```

Cloudvolume is officially supports on Linux and MacOS; Windows is community supported is known to create challenges during the installation process.

In [1]:
!pip install cloudvolume

In [2]:
import cloudvolume

## Neuropils 

The neuropils can be downloaded via cloudvolume. These are stored as an int -> mesh lookup. We provide a map to neuropil names.

In [3]:
np_path = "precomputed://gs://flywire_neuropil_meshes/neuropils/neuropil_mesh_v141_v5"
# For meshes used for the 630 release uncommented this path:
# np_path = "precomputed://gs://flywire_neuropil_meshes/neuropils/neuropil_mesh_v141_v3"
cv_nps = cloudvolume.CloudVolume(np_path)

mesh_map = {"0": "SCL_R", "1": "SMP_R", "2": "ME_R", "3": "CAN_L", "4": "MB_VL_L", "5": "FLA_R", "6": "LOP_L", "7": "IPS_L", "8": "EPA_L", "9": "PLP_R", "10": "MB_ML_L", "11": "BU_R", "12": "GOR_L", "13": "SPS_L", "14": "LO_R", "15": "SCL_L", "16": "GA_R", "17": "IB_R", "18": "ATL_L", "19": "CRE_R", "20": "LH_L", "21": "MB_CA_R", "22": "AOTU_L", "23": "ATL_R", "24": "AOTU_R", "25": "LAL_R", "26": "GNG", "27": "AL_R", "28": "MB_PED_R", "29": "AME_R", "30": "CRE_L", "31": "ICL_L", "32": "GOR_R", "33": "ICL_R", "34": "GA_L", "35": "EB", "36": "LOP_R", "37": "PVLP_R", "38": "IPS_R", "39": "PVLP_L", "40": "VES_R", "41": "MB_ML_R", "42": "SMP_L", "43": "ME_L", "44": "VES_L", "45": "AMMC_L", "46": "LAL_L", "47": "SLP_R", "48": "MB_PED_L", "49": "AVLP_R", "50": "WED_L", "51": "LO_L", "52": "EPA_R", "53": "PRW", "54": "LH_R", "55": "MB_VL_R", "56": "AME_L", "57": "AL_L", "58": "NO", "59": "PLP_L", "60": "WED_R", "61": "CAN_R", "62": "SLP_L", "63": "SIP_R", "64": "SPS_R", "65": "FB", "66": "MB_CA_L", "67": "IB_L", "68": "PB", "69": "AVLP_L", "70": "SAD", "71": "AMMC_R", "72": "SIP_L", "73": "BU_L", "74": "FLA_L", "75": "LA_R", "76": "LA_L", "77": "OCG"}

This command downloads all 78 neuropils and loads them into a dictionary

In [4]:
np_d = {
    v: cv_nps.mesh.get(int(k)) 
    for k, v in mesh_map.items() if int(k) < 78
}

Exists: 100%|█████████████████████████████████████| 1/1 [00:00<00:00,  7.35it/s]


Decoding Mesh Buffer: 100%|█████████████████████| 1/1 [00:00<00:00, 2187.95it/s]


Each mesh consists of vertices and faces. If you are interested in visualizing these meshes, checkout [MeshParty](https://meshparty.readthedocs.io/en/latest/).

## Point lookups in meshes

This section describes how one can lookup in which neuropil a point is located. It uses [ncollpyde](https://pypi.org/project/ncollpyde/). ncollpyde provides fast and convenient ways to lookup points with relation to meshes.

In [5]:
!pip install navis
!pip install ncollpyde

In [6]:
import navis 
import ncollpyde
import numpy as np

First, we need to transform the meshes into ncollpydes' format:

In [7]:
np_ncoll_d = {
    k: ncollpyde.Volume(m.vertices, m.faces, n_rays=3)
    for k, m in np_d.items()
}

The meshes use nanometers as resolution, so points need be given in nanometers as well. When copying points from neuroglancer, take note of the resolution given in the upper right.

In [8]:
example_locations = np.array([
    [480420, 308060,  97520],
    [530968, 161116, 159440]
])

Each neuropil needs to be independently tested:

In [9]:
volcon = {}
for k, vol in np_ncoll_d.items():
    volcon[k] = vol.contains(example_locations)

In [10]:
volcon

{'SCL_R': array([False, False]),
 'SMP_R': array([False, False]),
 'ME_R': array([False, False]),
 'CAN_L': array([False, False]),
 'MB_VL_L': array([False, False]),
 'FLA_R': array([False, False]),
 'LOP_L': array([False, False]),
 'IPS_L': array([False, False]),
 'EPA_L': array([False, False]),
 'PLP_R': array([False, False]),
 'MB_ML_L': array([False, False]),
 'BU_R': array([False, False]),
 'GOR_L': array([False, False]),
 'SPS_L': array([False, False]),
 'LO_R': array([False, False]),
 'SCL_L': array([False, False]),
 'GA_R': array([False, False]),
 'IB_R': array([False, False]),
 'ATL_L': array([False, False]),
 'CRE_R': array([False, False]),
 'LH_L': array([False, False]),
 'MB_CA_R': array([False, False]),
 'AOTU_L': array([False, False]),
 'ATL_R': array([False, False]),
 'AOTU_R': array([False, False]),
 'LAL_R': array([False, False]),
 'GNG': array([ True, False]),
 'AL_R': array([False, False]),
 'MB_PED_R': array([False, False]),
 'AME_R': array([False, False]),
 'CRE_L'